<img src="../idp.jpg">

# <p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Aula 7 - Pipeline</p>

## Objetivo do trabalho

Utilizar os conceitos de pipeline do sklearn e melhorar a legibilidade e manuntenção dos seus códigos de machine learning


## Fonte de dados

Link dos dados: https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data

Prever o valor de um imóvel não depende somente do bairro, da localização e do número de quartos. Envolve muito mais variáveis. 
Com 79 variáveis explicativas que descrevem (quase) todos os aspectos das casas residenciais em Ames, Iowa, esta competição desafia você a prever o preço final de cada casa.

A ideia deste notebook não é fazer uma regressão para adivinhar o preço da casa, mas sim aprender a trabalhar com o pipeline do sklearn. Mais info em: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html  

<img src="house.png">

## Como funciona o pipeline do sklearn?

A classe Pipeline é uma funcionalidade do Scikit-Learn que ajuda criar códigos que possuam um padrão que possa ser facilmente entendido e compartilhando entre times de cientistas e engenheiros de dados.

Códigos de machine-learning são melhores escritos através de pipelines, que em essência fazem o output de uma dada transformação nos dados se tornarem o input para uma outra transformação que será aplicada nos dados.

<img src="pipeline.jpg">

In [ ]:
# Importando as bibliotecas

import numpy as np 
import pandas as pd

## Lendo os datasets de treino e teste

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
# Verificando o shape do dataset
df_train.shape

In [ ]:
df_test.shape

## Limpeza de valores nulos

In [ ]:
df_train.isna().sum()

In [ ]:
df_test.isna().sum()

In [ ]:
df_train.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], axis=1)
df_test.drop(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], axis=1)

## Dividindo o dataset em treino e teste

In [ ]:
X_train = df_train.iloc[:, 1:-1] # ignora a coluna Id e Sale_price
X_test = df_test.iloc[:, 1:] # ignora a coluna Id
y_train = df_train.iloc[:, -1] # Apenas a coluna Sale_price

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

In [ ]:
y_train

## Selecione as colunas numéricas e categóricas

In [ ]:
num_features = X_train.dtypes[X_train.dtypes!='object'].index
cat_features = X_train.dtypes[X_train.dtypes=='object'].index

## Criar os transformers e o modelo

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

# Definindo o transformer para as variáveis numéricas
numerical_transformer = SimpleImputer(strategy='mean')

# Definindo o transformer para as variáveis categóricas
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), 
                                          ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))])



preprocessor = ColumnTransformer(transformers=[('numerical', numerical_transformer, num_features),
                                               ('categorical', categorical_transformer, cat_features)],
                                 remainder='passthrough')
# By specifying remainder='passthrough' , all remaining columns that were not specified in transformers will be automatically passed through. This subset of columns is concatenated with the output of the transformers.

# Setando o modelo /  estimador
model = RandomForestRegressor(n_estimators=1500)

## Criando efetivamente o pipeline e realizando .fit

In [ ]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                              ('model', model)])

my_pipeline.fit(X_train, y_train)

## Fazendo a predição

In [ ]:
y_pred = my_pipeline.predict(X_test)

In [ ]:
y_pred

## Gerando a saída com as predições dos valores dos imóveis

In [ ]:
output = pd.DataFrame({'Id': df_test.Id,
                       'Preço oficial': y_train.iloc[:-1].round(),
                      'Preço estimado': y_pred.round()})

In [ ]:
output

## Fim